In [1]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm 
import pandas as pd
from yahoo_fin.stock_info import get_data

## Scrape data from Yahoo

In [2]:
dfY = get_data('^VIX')['2014-01-02':'2021-09-20']
dfY.drop(['open', 'high', 'low', 'close', 'volume', 'ticker'],1,inplace=True)
dfY.reset_index(inplace = True)
dfY.columns = ['Trade Date', 'VIX']

dfY.head()

,Trade Date,VIX
0,2014-01-02,14.23
1,2014-01-03,13.76
2,2014-01-06,13.55
3,2014-01-07,12.92
4,2014-01-08,12.87


## Scrape data from CBOE

In [32]:
page=requests.get('https://www.cboe.com/us/futures/market_statistics/historical_data/')
soup = BeautifulSoup(page.text, 'html.parser')

dfs = soup.findAll('li', {'class':'mbn'})

In [33]:
# remove dfs of the year 2013 and the weekly ones VX + VXT01 ...
dfs = [df for df in dfs if not df.a.text.split('+')[1][3:5].isnumeric() and df.a['href'].split('/')[3].split('-')[0] != '2013']
len(dfs)

102

In [34]:
dicMonths = {'F':1,'G':2,'H':3,'J':4,'K':5,'M':6,'N':7,'Q':8,'U':9,'V':10,'X':11,'Z':12}
dicYears = {'2':2022, '1':2021, '0':2020, '9':2019, '8':2018, '7':2017, '6':2016, '5':2015, '4':2014, '3':2013}

#sort the names of dataframes by year and month
dff = [df.a.text.strip().split()[1] for df in dfs[::-1]]
sorted_dff = sorted(dff, key= lambda xy:(dicYears[xy[1]], dicMonths[xy[0]]))

#get the indexes of sorted dataframes
idx = [dff.index(elt) for elt in sorted_dff]

#get the sorted dataframes
dfs = dfs[::-1]
dfs = [dfs[i] for i in idx]

In [37]:
TradeDates = []
Name = []
Settle = []
DateExp = []


for df in tqdm(dfs) :
    text = df.a.text
    link = df.a['href']
    data = pd.read_csv('https://www.cboe.com/us/futures/market_statistics/historical_data/'+ link)
    TradeDates.extend(list(data['Trade Date'].values))
    Settle.extend(list(data['Settle'].values))
    Name.extend([text[-11:-9]] * data.shape[0])
    DateExp.extend([False] * (data.shape[0] - 1) + [True])
    
        

100%|████████████████████████████████████████████████████████████████████████████████| 102/102 [04:52<00:00,  2.87s/it]


In [38]:
dataframe = pd.DataFrame(list(zip(TradeDates, Name,  Settle, DateExp)),
               columns =['Trade Date', 'Name', 'Settle', 'DateExp'])

In [39]:
dataframe.head(1)

,Trade Date,Name,Settle,DateExp
0,2013-04-22,F4,0.0,False


## Data Preparation

In [40]:
dataframe['Trade Date'] = pd.to_datetime(dataframe['Trade Date'],errors='ignore')

In [41]:
feat = ['feature 1', 'feature 2', 'feature 3', 'feature 4', 'feature 5', 'feature 6', 'feature 7', 'feature 8', 'feature 9', 'feature 10']
data = pd.DataFrame(columns = feat)
data

,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10


In [42]:
new = dataframe.join(dfY.set_index('Trade Date'), on='Trade Date')
new = pd.concat([new, data])

new = new[new['VIX'].notna()]
#new=new.sort_values(ascending = False, by='Trade Date')

new.head()

,Trade Date,Name,Settle,DateExp,VIX,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10
177,2014-01-02,F4,14.20,False,14.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,2014-01-03,F4,14.05,False,13.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,2014-01-06,F4,13.90,False,13.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180,2014-01-07,F4,13.65,False,12.92,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
181,2014-01-08,F4,13.70,False,12.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
dicMonths = {'F':1,'G':2,'H':3,'J':4,'K':5,'M':6,'N':7,'Q':8,'U':9,'V':10,'X':11,'Z':12}
dicYears = {'2':2022, '1':2021, '0':2020, '9':2019, '8':2018, '7':2017, '6':2016, '5':2015, '4':2014}

def turnNameToDate(name) :
    m, y = name[0], name[1]
    m, y = dicMonths[m], dicYears[y]
    date = '-'.join([str(y), str(m)])
    return date
    
new['dateOfFile'] = new['Name'].apply(turnNameToDate)

In [44]:
new['ToCompareWith'] = new['Trade Date'].apply(lambda x: str(pd.to_datetime(x).year)+'-'+str(pd.to_datetime(x).month))

In [46]:
new['dateOfFile'] = pd.to_datetime(new['dateOfFile'],errors='ignore')
new['ToCompareWith'] = pd.to_datetime(new['ToCompareWith'],errors='ignore')
new.set_index('Trade Date', inplace = True)
new.head(2)

,Name,Settle,DateExp,VIX,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,dateOfFile,ToCompareWith
Trade Date,,,,,,,,,,,,,,,,
2014-01-02,F4,14.20,False,14.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,2014-01-01
2014-01-03,F4,14.05,False,13.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014-01-01,2014-01-01


In [47]:
uniqueMonthsYears = list(new.Name.unique())

In [48]:
dates = []

for date, name, tocomp, datefile, dateExp in zip(new.index[:], new.Name[:], new.ToCompareWith[:], new.dateOfFile[:], new.DateExp[:]) :

    a = pd.to_datetime(datefile) - pd.DateOffset(months=1)
    if (tocomp == datefile) | (str(tocomp).split()[0] == str(a).split()[0]) :
        if date not in dates and not dateExp : # 
            print(date)
            for i in range(1, 10) :
                feature_date = pd.to_datetime(date) + pd.DateOffset(months=i)
                idx = uniqueMonthsYears.index(name)
                try :
                    try :
                        values = new[new.index == str(feature_date).split()[0]]
                        value = values[values.Name == uniqueMonthsYears[idx+i]]['Settle'][0]
                        new.at[date, f'feature {i+1}']=value
                        
                    except :
                        values = new[new.index == date]
                        value = values[values.Name == uniqueMonthsYears[idx+i]]['Settle'][0]
                        new.at[date, f'feature {i+1}']=value

                except :
                    pass
            dates.append(date)
            

2014-01-02 00:00:00
2014-01-03 00:00:00
2014-01-06 00:00:00
2014-01-07 00:00:00
2014-01-08 00:00:00
2014-01-09 00:00:00
2014-01-10 00:00:00
2014-01-13 00:00:00
2014-01-14 00:00:00
2014-01-15 00:00:00
2014-01-16 00:00:00
2014-01-17 00:00:00
2014-01-21 00:00:00
2014-01-22 00:00:00
2014-01-23 00:00:00
2014-01-24 00:00:00
2014-01-27 00:00:00
2014-01-28 00:00:00
2014-01-29 00:00:00
2014-01-30 00:00:00
2014-01-31 00:00:00
2014-02-03 00:00:00
2014-02-04 00:00:00
2014-02-05 00:00:00
2014-02-06 00:00:00
2014-02-07 00:00:00
2014-02-10 00:00:00
2014-02-11 00:00:00
2014-02-12 00:00:00
2014-02-13 00:00:00
2014-02-14 00:00:00
2014-02-18 00:00:00
2014-02-19 00:00:00
2014-02-20 00:00:00
2014-02-21 00:00:00
2014-02-24 00:00:00
2014-02-25 00:00:00
2014-02-26 00:00:00
2014-02-27 00:00:00
2014-02-28 00:00:00
2014-03-03 00:00:00
2014-03-04 00:00:00
2014-03-05 00:00:00
2014-03-06 00:00:00
2014-03-07 00:00:00
2014-03-10 00:00:00
2014-03-11 00:00:00
2014-03-12 00:00:00
2014-03-13 00:00:00
2014-03-14 00:00:00


In [ ]:
new['feature 1'] = new['Settle']

In [66]:
new = new[new.DateExp != True]
b = new[new.ToCompareWith == new.dateOfFile]
bb = new[new.ToCompareWith == (new.dateOfFile - pd.DateOffset(months=1))]        

bbb = pd.concat([b, bb])

In [67]:
bbb.sort_values(by = ['Trade Date', 'dateOfFile'], inplace = True)
bbb.set_index('Trade Date', inplace = True)
bbb.head()

,Name,Settle,DateExp,VIX,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,dateOfFile,ToCompareWith
Trade Date,,,,,,,,,,,,,,,,
2014-01-02,F4,14.20,False,14.23,14.20,15.05,15.9,14.6,14.9,13.2,11.95,18.3,13.75,16.55,2014-01-01,2014-01-01
2014-01-02,G4,15.05,False,14.23,15.05,15.05,15.9,14.6,14.9,13.2,11.95,18.3,13.75,16.55,2014-02-01,2014-01-01
2014-01-03,F4,14.05,False,13.76,14.05,19.2,16.6,14.45,16.95,13.3,11.8,18.25,13.5,15.2,2014-01-01,2014-01-01
2014-01-03,G4,14.90,False,13.76,14.90,19.2,16.6,14.45,16.95,13.3,11.8,18.25,13.5,15.2,2014-02-01,2014-01-01
2014-01-06,F4,13.90,False,13.55,13.90,17.0,15.05,16.2,14.75,12.0,17.7,16.2,18.45,15.75,2014-01-01,2014-01-01


In [68]:
bbb = bbb[~bbb.index.duplicated(keep='first')]
bbb

,Name,Settle,DateExp,VIX,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10,dateOfFile,ToCompareWith
Trade Date,,,,,,,,,,,,,,,,
2014-01-02,F4,14.2000,False,14.230000,14.2000,15.05,15.9,14.6,14.9,13.2,11.95,18.3,13.75,16.55,2014-01-01,2014-01-01
2014-01-03,F4,14.0500,False,13.760000,14.0500,19.2,16.6,14.45,16.95,13.3,11.8,18.25,13.5,15.2,2014-01-01,2014-01-01
2014-01-06,F4,13.9000,False,13.550000,13.9000,17.0,15.05,16.2,14.75,12.0,17.7,16.2,18.45,15.75,2014-01-01,2014-01-01
2014-01-07,F4,13.6500,False,12.920000,13.6500,15.5,15.5,15.5,14.4,16.9,12.15,16.65,18.2,17.1,2014-01-01,2014-01-01
2014-01-08,F4,13.7000,False,12.870000,13.7000,14.45,15.25,15.15,14.45,16.8,12.4,16.0,13.35,15.25,2014-01-01,2014-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-14,U1,19.7150,False,19.459999,19.7150,21.5605,22.6707,23.3061,24.2993,24.7096,25.2,25.275,25.2,NaN,2021-09-01,2021-09-01
2021-09-15,V1,20.8379,False,18.180000,20.8379,22.2987,23.0433,24.0005,24.4179,24.9,25.0031,24.975,NaN,NaN,2021-10-01,2021-09-01
2021-09-16,V1,20.5556,False,18.690001,20.5556,22.0021,22.7954,23.85,24.2941,24.8,24.915,24.875,NaN,NaN,2021-10-01,2021-09-01


In [74]:
last.drop(['Name', 'Settle', 'DateExp', 'dateOfFile', 'ToCompareWith'], axis = 1, inplace = True)
last.head()

,VIX,feature 1,feature 2,feature 3,feature 4,feature 5,feature 6,feature 7,feature 8,feature 9,feature 10
Trade Date,,,,,,,,,,,
2014-01-02,14.23,14.20,15.05,15.9,14.6,14.9,13.2,11.95,18.3,13.75,16.55
2014-01-03,13.76,14.05,19.2,16.6,14.45,16.95,13.3,11.8,18.25,13.5,15.2
2014-01-06,13.55,13.90,17.0,15.05,16.2,14.75,12.0,17.7,16.2,18.45,15.75
2014-01-07,12.92,13.65,15.5,15.5,15.5,14.4,16.9,12.15,16.65,18.2,17.1
2014-01-08,12.87,13.70,14.45,15.25,15.15,14.45,16.8,12.4,16.0,13.35,15.25


In [ ]:
last.to_csv('res.csv')